## MisClassified Images for German on Winning Model
### Environment Intialization

In [1]:
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)

In [30]:
import tensorflow as tf
import os
import sys
IntialiseEnv()
import glob
import numpy as np
import cv2
import warnings
import pandas as pd 
import random
import re
import matplotlib.pyplot as plt
import import_ipynb

from Pre_Processing.Pre_Processing import FeatureExtraction

## Use GPU/ CPU Configuration 
Tensorflow version 2.0
Prompt to user if CPU/ GPU is in use with device name

In [3]:
# Use GPU/CPU Configurations

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
/device:GPU:0
Num GPUs Available:  1


In [5]:
def load_model():
    Model = tf.keras.models.load_model('../Model/Trained_Models/Belgium_Winning_98_97.h5')
    return Model

In [6]:
model= load_model()

In [7]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

### Retrieve Misclassified Images in German DataSet

In [8]:
## Read image in path
def readImage(path):
    img= cv2.imread(path)
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img= cv2.resize(img,(60,60))
    return img

In [9]:
## Function to get csv path
def getCsvPath(train_path_dir):
     return os.path.join(train_path_dir,list(filter(lambda x: '.csv' in x, os.listdir(train_path_dir)))[0])

In [10]:
# Function to get category directory and corresponding data frame
def getDirAndDataFrame(datasetDir,_dir):
    train_category_dir= os.path.join(datasetDir,_dir)
    train_csv_path= getCsvPath(train_category_dir)
    train_Data_frame = pd.read_csv(train_csv_path, delimiter=';')
    return train_category_dir, train_Data_frame

#### CreateMisClassifiedBatchFileCmd
Function creates .cmd bash file to copy all misclassified images in winning 
belgium model to MisClassified Images Folder

Resultant Image is Image_predictionofmodel_correctprediction.ppm

In [41]:
def CreateMisClassifiedBatchFileCmd():
    test_belgium_dataset_dir='../DataSet/BelgiumTSC_Testing/Testing/'
    f= open("misclassified_belgium.cmd","w+")
    for _dir in os.listdir(test_belgium_dataset_dir):
            print("Directory: ", _dir)
            # Get Directory for train with certain class ID and its associated dataframe
            test_category_dir, test_data_frame= getDirAndDataFrame(test_belgium_dataset_dir, _dir)
            # Get ClassID for entire category
            if len(test_data_frame['ClassId']) == 0:
                print("Skipping Directory "+ _dir + " No test images available")
                continue

            img_label=test_data_frame.iloc[0]['ClassId']

            for img_path in glob.glob(os.path.join(test_category_dir, '*.ppm')):

                #Load image in category path directory + Extraction of features
                img = readImage(img_path)
                img_features = np.array(FeatureExtraction(img))[:,:,np.newaxis]
                prediction= model.predict(img_features.reshape(1,60,60,1))
                if np.argmax(prediction) != img_label:
                    image= img_path.replace("\\",r"/")
                    image=image.replace(r"/",r"\\")

                    image_final= re.split(r"\\", img_path)[1][:-4]
                    image_final=image_final+"_"+str(np.argmax(prediction))+"_"+str(img_label)+".ppm"
                    f.write("copy "+image+ " MisClassification_Belgium_Images\\"+image_final +"\n")

    f.close()

In [42]:
CreateMisClassifiedBatchFileCmd()

Directory:  00000
Directory:  00001
Directory:  00002
Directory:  00003
Directory:  00004
Directory:  00005
Directory:  00006
Directory:  00007
Directory:  00008
Directory:  00009
Skipping Directory 00009 No test images available
Directory:  00010
Directory:  00011
Skipping Directory 00011 No test images available
Directory:  00012
Directory:  00013
Directory:  00014
Directory:  00015
Skipping Directory 00015 No test images available
Directory:  00016
Directory:  00017
Directory:  00018
Directory:  00019
Directory:  00020
Directory:  00021
Directory:  00022
Directory:  00023
Directory:  00024
Directory:  00025
Directory:  00026
Skipping Directory 00026 No test images available
Directory:  00027
Directory:  00028
Directory:  00029
Directory:  00030
Directory:  00031
Directory:  00032
Directory:  00033
Skipping Directory 00033 No test images available
Directory:  00034
Directory:  00035
Directory:  00036
Skipping Directory 00036 No test images available
Directory:  00037
Directory:  0003